# More Data Preprocessing (with Pandas)

Pandas is a very useful data analytics package within Python.

Let's start with some random data.

In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'], columns=['one', 'two', 'three'])
df

,one,two,three
a,0.581344,-0.298158,0.843908
c,-1.617379,0.083731,-0.748585
e,1.449528,-0.197890,0.165317
f,0.027281,-0.196457,-0.988407
h,-0.042482,0.565191,-0.763581


We know with Pandas we can mix & match data types, so let us add two more columns, named 'four' and 'five':

In [49]:
df['four'] = 'bar'
df['five'] = df['one'] > 0
df

,one,two,three,four,five
a,0.581344,-0.298158,0.843908,bar,True
c,-1.617379,0.083731,-0.748585,bar,False
e,1.449528,-0.197890,0.165317,bar,True
f,0.027281,-0.196457,-0.988407,bar,True
h,-0.042482,0.565191,-0.763581,bar,False


Adding new rows is also simple. Below we include three extra empty rows:

In [50]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df2

,one,two,three,four,five
a,0.581344,-0.298158,0.843908,bar,True
b,NaN,NaN,NaN,NaN,NaN
c,-1.617379,0.083731,-0.748585,bar,False
d,NaN,NaN,NaN,NaN,NaN
e,1.449528,-0.197890,0.165317,bar,True
f,0.027281,-0.196457,-0.988407,bar,True
g,NaN,NaN,NaN,NaN,NaN
h,-0.042482,0.565191,-0.763581,bar,False


# Your Turn Here

Do you still remember how to index a row/column?

In [51]:
#### index row 'c' below
df.loc['c']

one       -1.61738
two      0.0837313
three    -0.748585
four           bar
five         False
Name: c, dtype: object

In [52]:
#### index column 'two' below
df['two']

a   -0.298158
c    0.083731
e   -0.197890
f   -0.196457
h    0.565191
Name: two, dtype: float64

pandas has two functions isnull() and notnull() that return boolean objects when called.

In [53]:
# False when the df has a number, true when it is null

pd.isnull(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [54]:
# Opposite of .isnull()
# True when the df row is filled in

pd.notnull(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

Missing values propagate naturally through arithmetic operations between pandas objects.

In [55]:
a = df[['one','two']]
a['one']['a':'e'] = float('nan')
a

,one,two
a,NaN,-0.298158
c,NaN,0.083731
e,NaN,-0.197890
f,0.027281,-0.196457
h,-0.042482,0.565191


In [56]:
b = df[['one','two','three']]
b

,one,two,three
a,0.581344,-0.298158,0.843908
c,-1.617379,0.083731,-0.748585
e,1.449528,-0.197890,0.165317
f,0.027281,-0.196457,-0.988407
h,-0.042482,0.565191,-0.763581


In [57]:
# When df a and b are added together, I expected it to add the numbers in the columns
# All the NaN in row 1 came back, rather than the numbers in table a, and added row f and h 
# Column 'three' returned all NaN.. because there is no column 'three' in df a ?
# Column 'two' added all the numbers together from both df a and b

a + b

,one,three,two
a,NaN,NaN,-0.596315
c,NaN,NaN,0.167463
e,NaN,NaN,-0.395780
f,0.054563,NaN,-0.392913
h,-0.084964,NaN,1.130383


# How to deal with Missing Values

## Deleting Missing Values

The simplest method is always dropping all missing values - but it is highly **discouraged!!!**

In [58]:
a['one'].dropna()

f    0.027281
h   -0.042482
Name: one, dtype: float64

By default, dropna() will drop any row containing **NaN** values, but you can change that by using the *axis=* and *thresh=* arguments.

**NOTE**: Dropping rows or columns have different uses.

In [59]:
#### This statement drops any column with NaN values
a.dropna(axis=1)

# column one and three dropped

,two
a,-0.298158
c,0.083731
e,-0.197890
f,-0.196457
h,0.565191


In [60]:
#### thresh determines how many non-NaN values a column/row should have without being dropped
c = a + b
c.dropna(axis=1, thresh=2)

# thresh: since column 'one' has 2 values, it is not dropped, but 'three' is

,one,two
a,NaN,-0.596315
c,NaN,0.167463
e,NaN,-0.395780
f,0.054563,-0.392913
h,-0.084964,1.130383


You can also use the *how=* argument to determine how do you want to remove the NaN values.

In [61]:
#### By default, dropna() drops column/row with any NaN values
#### how = 'all' changes that to dropping column/row with all NaN values
c.dropna(axis=1, how='all')

# 'all' = drops a column with no values

,one,two
a,NaN,-0.596315
c,NaN,0.167463
e,NaN,-0.395780
f,0.054563,-0.392913
h,-0.084964,1.130383


You can refer to the [pandas.dropna() docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) for more information.

## Imputing Missing Values

Imputing means filling missing values - you can do that when the missing and non-missing values are in some type of relationship.

In [62]:
my_series = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
my_series

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [63]:
#### you can fill missing values with a specific value (0)
my_series.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Alternatively, we can specify a forward-fill to propagate the previous value forward:

In [64]:
# forward-fill
my_series.fillna(method='ffill')

# b is filled with value from a, and d is filled with value from c

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [65]:
my_series.fillna(method='bfill')

# b is filled with value from c, and d is filled with value from e

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

## Your Turn Here
Aforementioned method can be applied to dataframes. 

In [66]:
#### Let us generate a random dataframe
rand_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['one', 'two', 'three'])
rand_df = rand_df.mask(np.random.random(rand_df.shape) < .3)
rand_df

,one,two,three
a,0.856080,0.597081,-0.432356
b,-0.345258,NaN,-0.580196
c,-0.378637,NaN,1.016314
d,-1.559417,0.605380,-1.295018
e,-0.463456,2.449652,-0.225985


Your tasks are as follows:

- fill missing values in column **one** with value 1;
- fill missing values in column **two** with forward-filling;
- fill missing values in column **three** with backward-filling.

In [67]:
#### insert your code here

# 1. Fill missing values in column one with value 1

rand_df['one'].fillna(1, inplace=True)

# 2. Fill missing values in column two with forward-filling

rand_df['two'].fillna(method='ffill', inplace=True)

# 3. Fill missing values in column three with backward-filling

rand_df['three'].fillna(method='bfill', inplace=True)

rand_df

,one,two,three
a,0.856080,0.597081,-0.432356
b,-0.345258,0.597081,-0.580196
c,-0.378637,0.597081,1.016314
d,-1.559417,0.605380,-1.295018
e,-0.463456,2.449652,-0.225985


A useful approach for imputing your missing data is to use mean/mode to replace missing data - the reason behind this logic is that if we are going to *guess* the values of the missing data, the highest chance would be guessing it to be the mean/mode if the data follows **normal** distribution.

In [68]:
#### let us generate another DF
my_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['A', 'B', 'C'])
my_df = my_df.mask(np.random.random(my_df.shape) < .3)
my_df

,A,B,C
a,0.701132,0.949419,-0.407764
b,-0.092103,1.305671,NaN
c,0.921270,1.101752,-1.594850
d,2.034287,0.831961,-0.184860
e,-0.242945,NaN,0.141744


In [69]:
#### Let us check if there is any missing value in the df
my_df.isnull().values.any()

# True = there are missing values

True

In [70]:
#### Then we are getting the mean of the DF
#### Note that since we only 
my_df.mean()

# Mean of whatever values are in the columns

A    0.664328
B    1.047201
C   -0.511432
dtype: float64

In [71]:
df_filled = my_df.fillna(my_df.mean())
df_filled

# Replaces missing values with the mean

,A,B,C
a,0.701132,0.949419,-0.407764
b,-0.092103,1.305671,-0.511432
c,0.921270,1.101752,-1.594850
d,2.034287,0.831961,-0.184860
e,-0.242945,1.047201,0.141744


In [72]:
#### Now let us check again if there is any missing values
df_filled.isnull().values.any()

# False = no missing values

False

More info regarding how to handle missing data can be found [here](https://machinelearningmastery.com/handle-missing-data-python/).

# Other Tasks in Data Preprocessing

- Handling categorical data (coding)
- Handling imbalanced data
- feature engineering

These topics will be covered in later part of this class.